In [1]:
from app.models import Comment

In [2]:
class PostCommentPathDetails:
    """" An object containing comment path details for a post """
    def __init__(self, post_id):
        self.id = post_id

    @property
    def full_comment_tree_list(self):
        return self._get_comment_tree_list()

    @property
    def comment_tree_list(self):
        return self._get_pruned_list(self.full_comment_tree_list)

    @property
    def levels(self):
        return self._get_levels()

    def _get_path(self, comment_id, path=None):
        """ Gets a comment's tree path from its ID """
        comment = Comment.query.filter_by(id=comment_id).first()
        if path is None:  # Create path list on first pass
            path = list()
        if comment.id not in path:  # Add initial comment_id to path
            path.append(comment.id)
        if comment.parent_id:  # Add parent_id to path
            path.append(comment.parent_id)
            return self._get_path(comment.parent_id, path=path)  # Loop
        else:
            path.reverse()  # Flip list so it begins at root
            return path

    def _get_comment_tree_list(self):
        """ Returns a full list of comment tree paths """
        comments = Comment.query.filter_by(post_id=self.id).all()
        comment_tree_list = list()
        for comment in comments:
            comment_path = self._get_path(comment.id)
            comment_tree_list.append(comment_path)
        return comment_tree_list

    def _check_sublist(self, sublist, full_list):
        """
        Checks if comment tree exists as a subset 
        of any other comment tree on the post
        """
        check = False
        for item in full_list:
            if set(sublist).issubset(set(item)):
                check = True
                return check
            else:
                pass

        return check

    def _get_pruned_list(self, input_tree_list, pruned_list=None):
        """ Removes redundant paths from list of comment tree paths """
        tree_list = input_tree_list.copy()  # Copy tree list to work with
        if pruned_list == None:  # Create pruned path list on first pass
            pruned_list = list()
        if len(tree_list) > 0:
            for tree in tree_list:
                tree_list.remove(tree)
                # Check if tree is part of a larger, unique tree in the list
                if not self._check_sublist(tree, tree_list):
                    pruned_list.append(tree)  # Append full unique tree
                    return self._get_pruned_list(  # Loop
                        tree_list, pruned_list=pruned_list)
                else:
                    return self._get_pruned_list(  # Loop
                        tree_list, pruned_list=pruned_list)
        else:  # Return when list has been exhausted
            return pruned_list

    def _get_levels(self):
        """ Returns the level count of the post's comments """
        return len(max(self.comment_tree_list, key=len))

    def get_depth_index(self, comment_id):
        """ Returns the depth (level index) of the comment """
        return (len(self._get_path(comment_id)) - 1)

    def get_level(self, n, parent_id=None, root_id=None):
        """
        Gets all comments at level 'n'.

        Adding the 'parent_id=<PARENT-COMMENT-ID>' argurment will get only
            comments stemming from that parent at level 'n'.

        Adding the 'root_id=<ROOT-COMMENT-ID>' argument will get only
            comments at level 'n' from the root comment.
        """
        level = list()
        for i in self.comment_tree_list:
            # Parent ID argument check
            if parent_id and i[parent_id] \
                and len(i) >= (n + 1) \
                and i[n] not in level:  # Ignore duplicates
                level.append(i[n])
            # Root ID argument check
            if root_id and i[0] == root_id and len(i) >= (n + 1) \
                and i[n] not in level:  # Ignore duplicates
                level.append(i[n])
            # No argument check
            elif root_id is None and len(i) >= (n + 1) \
                and i[n] not in level:  # Ignore duplicates
                level.append(i[n])
            else:
                pass
        if len(level) >= 1:
            return level
        else:
            return None
    
    def get_children(self, comment_id, paths=False):
        """ Returns a list of a comment's children or full child paths """
        child_depth = self.get_depth_index(comment_id) + 1
        children = list()
        for i in self.comment_tree_list:
            if comment_id in i and not paths and child_depth < len(i):  # Child items only
                if i[child_depth] not in children:  # Remove duplicates
                    children.append(i[child_depth])
                else:
                    pass
            elif comment_id in i and paths and child_depth < len(i):  # Full child paths
                child_path = i[child_depth::]
                children.append(child_path)
            else:
                pass
        if len(children) > 0:
            return children
        else:
            return None
    
    def get_parent(self, comment_id):
        comment = Comment.query.filter_by(id=comment_id).first()
        return comment.parent_id
        
    def _make_child_nodes(self, id, child_node=None):
        """
        Creates list containing child node dicts
        Returns an empty list if reference node has no children
        """
        if not child_node:
            child_node = list()
        children = self.get_children(id)  # Get child comment IDs
        if children:
            for child in children:
                cnode_children = self.get_children(child)  # Get child comment's children
                if cnode_children:
                    for c in cnode_children:
                        c_children = self._make_child_nodes(c, child_node=child_node)
                        child_node.append(self._node(child, children=c_children))
                else:
                    child_node.append(self._node(child))
            return child_node
        else:
            return []

    def _node(self, id, children=[]):
        node = {'id': id, 'children': children}
        return node

In [3]:
pcpd = PostCommentPathDetails(1)

Return all comment IDs by given depth level index

In [4]:
print(f'LEVEL 0: {pcpd.get_level(0)} | LEVEL 1: {pcpd.get_level(1)} | LEVEL 2: {pcpd.get_level(2)}')

LEVEL 0: [1, 10] | LEVEL 1: [4, 3, 2] | LEVEL 2: [5, 6, 7, 8, 9]


Return comment's depth index

In [5]:
pcpd.get_depth_index(4)

1

Get comment's direct children

In [6]:
pcpd.get_children(4)

[5, 6, 7]

Get comment's childrens' paths

In [7]:
pcpd.get_children(1, paths=True)

[[4, 5], [4, 6], [4, 7], [3, 8], [2, 9]]

Return every full comment path

In [9]:
pcpd.comment_tree_list

[[1, 4, 5], [1, 4, 6], [1, 4, 7], [1, 3, 8], [1, 2, 9], [10]]

Create child nodes for given comment -- THIS NEEDS TO BE FIXED --

In [10]:
nodes = pcpd._make_child_nodes(1)

Need to add rule to omit duplicate child items and get rucursion working so it adds child comment's children

In [11]:
print(nodes)

[{'id': 4, 'children': []}, {'id': 4, 'children': []}, {'id': 4, 'children': []}, {'id': 3, 'children': []}, {'id': 2, 'children': []}]
